# load results

In [38]:
import json


def load_jsonl(file_path: str) -> list:
    assert file_path.endswith('.jsonl') or file_path.endswith('.json')
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

results = load_jsonl('./output/l1b_ft_test.json')
results[0]

{'id': 0,
 'problem': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'solution': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\nThe answer is: 18',
 'generated_solution': ' \nJanet lays 16 eggs per day and she eats 3 for breakfast, so she has 16 - 3 = 13 eggs left for the market each day.\nShe sells 13 eggs at the market and makes 13 x 2 = $26 from the eggs. \nThe answer is: 26.'}

In [39]:
import re

def get_aug_answer(full_answer):
    full_answer = full_answer.lower()
    idx = full_answer.rfind("the answer is")
    if idx == -1:
        return None
    else:
        answer = full_answer[idx + len("the answer is: "):]
        answer = answer.replace(":", "").replace("$", "").strip()
        if len(answer)> 0:
            if answer[-1] == ".":
                answer = answer[:-1]
            left = "\\boxed{"
            if answer[:len(left)] == left and answer[-1] == "}":
                answer = answer[len(left):-1]
        return answer.replace(",", "")


def calculate_acc(responses, gt_solutions):
    correct = 0
    total = len(gt_solutions)
    
    for resp, sol in zip(responses, gt_solutions):
        gt_final_solution = get_aug_answer(sol)
        response_final_solution = get_aug_answer(resp)
        
        if response_final_solution is None:
            continue

        if response_final_solution == gt_final_solution:
            correct += 1
                
            

    return correct / total

calculate_acc([r['generated_solution'] for r in results], [r['solution'] for r in results])

0.17437452615617893